In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import FinanceDataReader as fdr

df1 = pd.read_csv('tr_basic_cardl_20210101_20230630.csv') #롯데카드
df2 = pd.read_csv('tr_basic_cardh_20190101_20230630.csv') #하나카드
df3 = pd.read_csv('LIST_BT.csv') #종목코드

lottecard = df1[df1['mandata_brand_name'].isin(['MEGASTUDYCO LTD_ALL', 'DIGITAL DAESUNG CO LTD_ALL'])]
hanacard = df2[df2['mandata_brand_name'].isin(['MEGASTUDYCO LTD_ALL', 'DIGITAL DAESUNG CO LTD_ALL'])]
card_data = pd.merge(lottecard, hanacard, on=['transaction_date', 'mandata_brand_name'], how='outer')
card_data.to_csv('card_data_(daesung,mega).csv', index=False)

In [6]:
data = pd.read_csv('card_data_(daesung,mega).csv', parse_dates=['transaction_date'])
data['mon'] = data['transaction_date'].dt.month
data['year'] = data['transaction_date'].dt.year
monthly_data = data.groupby(['mandata_brand_name', 'year', 'mon']).agg({
    'sales_l': 'sum',
    'number_l': 'sum',
    'users_l': 'sum',
    'sales_h': 'sum',
    'number_h': 'sum',
    'users_h': 'sum'
}).reset_index()
yearly_data = data.groupby(['mandata_brand_name', 'year']).agg({
    'sales_l': 'sum',
    'number_l': 'sum',
    'users_l': 'sum',
    'sales_h': 'sum',
    'number_h': 'sum',
    'users_h': 'sum'
}).reset_index()
monthly_data['sales_all_month'] = monthly_data['sales_l'] + monthly_data['sales_h']
yearly_data['sales_all_year'] = yearly_data['sales_l'] + yearly_data['sales_h']
monthly_data = pd.merge(monthly_data, yearly_data[['mandata_brand_name', 'year', 'sales_all_year']], on=['mandata_brand_name', 'year'], how='left', suffixes=('', '_right'))

monthly_data.to_csv('sales_data_(daesung,mega).csv', index=False)

In [7]:
df = pd.read_csv('sales_data_(daesung,mega).csv')

#연매출비교
dfsaleyear = df.groupby(['year', 'mandata_brand_name'])['sales_all_year'].first().reset_index()
total_sales_by_year = dfsaleyear.groupby('year')['sales_all_year'].transform('sum')
dfsaleyear['yearly_ratio'] = dfsaleyear['sales_all_year'] / total_sales_by_year
yearly_ratio = dfsaleyear.copy()
yearly_ratio['year'] += 1

#월매출비교
dfsalemon = df.groupby(['year', 'mon', 'mandata_brand_name'])['sales_all_month'].sum().reset_index()
total_sales_by_mon_year = dfsalemon.groupby(['mon', 'year'])['sales_all_month'].sum()
dfsalemon['monthly_ratio'] = dfsalemon.apply(lambda row: row['sales_all_month'] / total_sales_by_mon_year.loc[(row['mon'], row['year'])], axis=1)
monthly_ratio = dfsalemon.copy()

for i in range(len(monthly_ratio)):
    year = monthly_ratio.loc[i, 'year']
    mon = monthly_ratio.loc[i, 'mon']
    name = monthly_ratio.loc[i, 'mandata_brand_name']
    
    yearly_ratio_value = yearly_ratio[(yearly_ratio['year'] == year) & (yearly_ratio['mandata_brand_name'] == name)]['yearly_ratio'].values
    monthly_ratio_value = monthly_ratio[(monthly_ratio['year'] == year) & (monthly_ratio['mon'] == mon) & (monthly_ratio['mandata_brand_name'] == name)]['monthly_ratio'].values

    if yearly_ratio_value.size > 0 and monthly_ratio_value.size > 0:
        if monthly_ratio_value[0] > yearly_ratio_value[0]:
            monthly_ratio.loc[i, 'active'] = 'buy'
        elif monthly_ratio_value[0] < yearly_ratio_value[0]:
            monthly_ratio.loc[i, 'active'] = 'sell'
        else:
            monthly_ratio.loc[i, 'active'] = 'hold'
    else:
        monthly_ratio.loc[i, 'active'] = 'no data'

monthly_ratio.to_csv('monthly_active_(daesung,mega).csv', index=False)

monthly_ratio = monthly_ratio[monthly_ratio['active'] != 'no data']
sell_previous = monthly_ratio[monthly_ratio['active'] == 'buy']
sell_current = monthly_ratio[monthly_ratio['active'] == 'sell']
merged = sell_current.merge(sell_previous, how='outer', indicator=True)
changed_rows = merged[merged['_merge'] == 'right_only']
changed_rows = changed_rows.drop(columns=['_merge'])

changed_rows.to_csv('table_(daesung,mega).csv', index=False)



In [12]:
m_close = pd.read_csv('m_close.csv', parse_dates=['transaction_date'])
d_close = pd.read_csv('d_close.csv', parse_dates=['transaction_date'])
close_dm = pd.merge(d_close, m_close[['transaction_date', 'Close']], on='transaction_date', how='left', suffixes=('_d', '_m'))
close_dm = pd.read_csv('stock_data_(daesung,mega).csv', index_col='transaction_date', parse_dates=True)

start_date = '2020-01-31'
end_date = '2020-11-30'

sell_d = close_dm.loc[start_date:end_date]
sell_d['close_d_return'] = sell_d['Close_d'].pct_change()
sell_d['close_m_return'] = sell_d['Close_m'].pct_change()
sell_d['close_d_return'] = sell_d['close_d_return'].ffill()
sell_d['close_m_return'] = sell_d['close_m_return'].ffill()
sell_d['cumulative_d_return'] = (1 + sell_d['close_d_return']).cumprod() - 1
sell_d['cumulative_m_return'] = (1 + sell_d['close_m_return']).cumprod() - 1
sell_d['d_return_sell'] = -sell_d['cumulative_d_return']

sell_d['return'] = sell_d['cumulative_m_return'] + sell_d['d_return_sell']
sell_d.to_csv('close_dm1.csv', index=True)

print("메가스터디 롱 누적수익률:", sell_d['cumulative_m_return'].iloc[-1]*100)
print("디지털대성 숏 누적수익률:", sell_d['d_return_sell'].iloc[-1]*100)
print("매매 누적수익률:", sell_d['return'].iloc[-1]*100)

return1 = sell_d['return'].iloc[-1]

,transaction_date,Close_d,Close_m
0,2021-02-01,7570,10750
1,2021-02-02,7490,10950
2,2021-02-03,7380,11000
3,2021-02-04,7350,11000
4,2021-02-05,7400,11050
...,...,...,...
613,2023-07-25,5960,10090
614,2023-07-26,5880,9930
615,2023-07-27,5910,10300
616,2023-07-28,5810,10250


In [ ]:
start_date = '2020-11-30'
end_date ='2022-02-28'
sell_m = close_dm.loc[start_date:end_date]

sell_m['close_d_return'] = sell_m['Close_d'].pct_change()
sell_m['close_m_return'] = sell_m['Close_m'].pct_change()
sell_m['close_d_return'] = sell_m['close_d_return'].ffill()
sell_m['close_m_return'] = sell_m['close_m_return'].ffill()
sell_m['cumulative_d_return'] = (1 + sell_m['close_d_return']).cumprod() - 1
sell_m['cumulative_m_return'] = (1 + sell_m['close_m_return']).cumprod() - 1
sell_m['m_return_sell'] = -sell_m['cumulative_m_return']
sell_m['return'] = sell_m['cumulative_d_return'] + sell_m['m_return_sell']+return1
sell_m.to_csv('close_dm2.csv', index=True)

print("메가스터디 숏 누적수익률:", sell_m['m_return_sell'].iloc[-1]*100)
print("디지털대성 롱 누적수익률:", sell_m['cumulative_d_return'].iloc[-1]*100)
print("매매 누적수익률:", sell_m['return'].iloc[-1]*100)

return1 = sell_m['return'].iloc[-1]

참고들

In [ ]:
close_dm = pd.read_csv('card_data_(daesung,mega).csv', parse_dates=['transaction_date'])
close_dm = close_dm[['transaction_date', 'mandata_brand_name']]
m_df = close_dm[close_dm['mandata_brand_name'] == 'MEGASTUDYCO LTD_ALL']
d_df = close_dm[close_dm['mandata_brand_name'] == 'DIGITAL DAESUNG CO LTD_ALL']

m_df = m_df.sort_values(by='transaction_date')
d_df = d_df.sort_values(by='transaction_date')

next_month_date = m_df['transaction_date'].iloc[-1] + pd.offsets.MonthBegin(1)

new_data = {
    'transaction_date': pd.date_range(start=next_month_date, periods=31),
    'mandata_brand_name': ['MEGASTUDYCO LTD_ALL'] * 31
}
new_data2 = {
    'transaction_date': pd.date_range(start=next_month_date, periods=31),
    'mandata_brand_name': ['DIGITAL DAESUNG CO LTD_ALL'] * 31
}

new_rows_df = pd.DataFrame(new_data)
new_rows_df2 = pd.DataFrame(new_data2)
m_df = pd.concat([m_df, new_rows_df], ignore_index=True)
d_df = pd.concat([d_df, new_rows_df2], ignore_index=True)

In [ ]:
m_close = fdr.DataReader('072870', '2021-02-01', '2023-07-31') 
m_close = m_close[['Close']]
d_close = fdr.DataReader('068930', '2021-02-01', '2023-07-31') 
d_close = d_close[['Close']]
m_close.to_csv('m_close.csv', index=True)
d_close.to_csv('d_close.csv', index=True)

h_close = fdr.DataReader('069960', '2019-01-31', '2023-07-31') 
h_close = h_close[['Close']]
l_close = fdr.DataReader('023530', '2019-01-31', '2023-07-31') 
l_close = l_close[['Close']]
h_close.to_csv('h_close.csv', index=True)
l_close.to_csv('l_close.csv', index=True)

e_close = fdr.DataReader('139480', '2019-01-31', '2023-07-31') 
e_close = e_close[['Close']]
s_close = fdr.DataReader('004170', '2019-01-31', '2023-07-31') 
s_close = s_close[['Close']]
e_close.to_csv('e_close.csv', index=True)
s_close.to_csv('s_close.csv', index=True)